# GPT 🤯

This notebook helps you grab the idea behind GPT model. 🔥

---

Almost everything was proudly nicked from [Andrej Karpathy](https://github.com/karpathy) and is based on his [amazing video](https://www.youtube.com/watch?v=kCc8FmEb1nY)

---

This notebooks tries to explain some concepts and ideas in neural network modeling. It's ultimate goal is to reimplement [nanoGPT](https://github.com/karpathy/nanoGPT) - beginner-friendlyish repository, which can be summarized as follows:
![](https://github.com/karpathy/nanoGPT/raw/master/assets/nanogpt.jpg)

---

A couple of words about GPT, before getting started:

- OG paper is [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- Decoder-only part
- OpenAI [GPT-3 paper](https://arxiv.org/abs/2005.14165)
- OpenAI [ChatGPT blogpost](https://openai.com/blog/chatgpt)
- Mind-boggling [GPT4 blogpost](https://openai.com/product/gpt-4)

Only requirements needed is [pytorch](https://pytorch.org)

It can be installed via `pip install torch`

In case you don't feel confident, check out [torch's tutorials](https://pytorch.org/tutorials/)

In [2]:
#!pip install torch  # in colab, it's already installed

In [3]:
import torch

## Data

First of all, dataset has to be defined. The choice is [shakespeare-dataset](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)

Download the dataset, define some helper function. Some preparations need to be arranged.

In [8]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [9]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [10]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [12]:
# let's look at the first 300 characters
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [13]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(repr("".join(chars)))
print(vocab_size)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
65


Prepare functions for [tokenization](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html)

For the sake of simplicity per-character tokenization is used.
In real-world models [Byte Pair encoding](https://en.wikipedia.org/wiki/) is the most popular choice.

Here is the [OpenAI's implementation](https://github.com/openai/tiktoken)

In [14]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [15]:
import torch


data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:300]) # the 300 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [16]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
    print(f"decoded input is {decode(context.numpy())} the target: {decode([target.tolist()])}")
    print("-"*10)

when input is tensor([18]) the target: 47
decoded input is F the target: i
----------
when input is tensor([18, 47]) the target: 56
decoded input is Fi the target: r
----------
when input is tensor([18, 47, 56]) the target: 57
decoded input is Fir the target: s
----------
when input is tensor([18, 47, 56, 57]) the target: 58
decoded input is Firs the target: t
----------
when input is tensor([18, 47, 56, 57, 58]) the target: 1
decoded input is First the target:  
----------
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
decoded input is First  the target: C
----------
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
decoded input is First C the target: i
----------
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
decoded input is First Ci the target: t
----------


In [22]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [23]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## Baseline: Bigram model

We'll build a simple baseline around [Bigram model](https://web.stanford.edu/~jurafsky/slp3/3.pdf) and then will iterate over it, until we reach GPT!!!

There is a great [video](https://www.youtube.com/watch?v=PaCmpygFfXo&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=2) by Andrej Karpathy. You should totaly check this out

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.7658, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [28]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [29]:
batch_size = 32
log_interval = 1000

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps % log_interval == 0:
        print(loss.item())

print("-"*10)
print(loss.item())

4.704006195068359
3.7031264305114746
3.137178659439087
2.776794672012329
2.5844571590423584
2.5105183124542236
2.531585931777954
2.504757881164551
2.4696712493896484
2.4838879108428955
----------
2.5727508068084717


In [29]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

## Self Attention

Now we gonna dive deep into the self-attention mechanism.

The full architecture schema is:

![](https://production-media.paperswithcode.com/social-images/oVEwwksZyfDziYzq.png)


It's greatly encouraged to check out [the paper](https://arxiv.org/abs/1706.03762)

We'll start with *Multi-Head Attention* block

### The mathematical trick in self-attention

Bigram model is simple but not brilliant. The main drawback is it checks only the previous token to generate a new one.

So, it's kinda intuitive to aggregate information not only from the last token, but all previous.

Let's try aggregate it by averaging all previous tokens (this is not really brilliant idea, but we need something to start with): for each token, take an average of all previous tokens and this token.

For example, for token number 3:
- take tokens from the positions 0, 1, 2, 3
- take an average

Actually, this method is called BagOfWords

In [31]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [32]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [35]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [36]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [38]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / a.sum(dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [39]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [43]:
t = 3
tril = torch.tril(torch.ones(t, t))
wei = torch.zeros((t, t))
wei

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [44]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [45]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [46]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

Now, let's do these averaging data-dependent. And that will be self-attention mechanism exactly!

In [47]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# key - "what I can give you"
# query - "what I look for"
# value - "my representation"
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)

wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [49]:
# scaling factor
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)

print(f"k var: {k.var()}")
print(f"q var: {q.var()}")
print(f"wei var: {wei.var()}")

k var: 0.9005609154701233
q var: 1.003690242767334
wei var: 15.931692123413086


In [50]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

print(f"k var: {k.var()}")
print(f"q var: {q.var()}")
print(f"wei var: {wei.var()}")

k var: 1.0966005325317383
q var: 0.9415779113769531
wei var: 1.0064778327941895


In [51]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [52]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

Almost done!


Look at the actual self-attention layer. It looks almost the same, right? There is some stuff to be done, though.

![](https://production-media.paperswithcode.com/method_collections/22f26a20-1608-449b-ab55-55d8136e097e.png)



**TODO list:**
- add [LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
- add non-linearity (originally, [GELU](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) is used)
- add [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)
- make multiple heads
- residual connections

[Attention Is All You Need](https://arxiv.org/abs/1706.03762)

![](https://production-media.paperswithcode.com/methods/multi-head-attention_l1A3G7a.png)

First, let's finish with the self-attention module - add multiple heads!

And, to be like a real researcher, we'll wrap it in `nn.Module`

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, n_embd=128):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, n_embd=128):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


What we've done is quite inefficient. For a better implementation check [here](https://github.com/karpathy/nanoGPT/blob/master/model.py)

Or just use [MultiheadAttention module](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)

Now, let's talk all other modules:

1. LayerNorm

![](https://pytorch.org/docs/stable/_images/layer_norm.jpg)

[paper](https://arxiv.org/abs/1607.06450)

In [57]:
class LayerNorm:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    shape = [i for i in range(len(x.shape))][1:]
    xmean = x.mean(shape, keepdim=True)
    xvar = x.var(shape, keepdim=True)
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm((10, 32))
x = torch.randn(32, 10, 32)
x = module(x)
x.shape

torch.Size([32, 10, 32])

2. GELU

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/ReLU_and_GELU.svg/1200px-ReLU_and_GELU.svg.png)

[paper](https://arxiv.org/abs/1606.08415)

In [58]:
import math

def gelu(x):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

3. Dropout

![](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-23_at_6.19.24_PM.png)

[paper](https://arxiv.org/abs/1207.0580)

In [72]:
class Dropout:
    def __init__(self, p=0.5):
        # p - probability to be zero-out
        self.p = p
    
    def __call__(self, x, is_training=True):
        if is_training:
            probs = torch.ones_like(x) * self.p
            mask = 1. - torch.bernoulli(probs)
            print(mask)
            x = x * mask
            if self.p != 1.0:
                return x / (1. - self.p)  # normalization
            return x
        else:
            return x


torch.manual_seed(1337)
dropout = Dropout(0.5)
x = torch.randn(3, 5)
print(x)
print(dropout(x))

tensor([[-2.0260, -2.0655, -1.2054, -0.9122, -1.2502],
        [ 0.8032, -0.2071,  0.0544,  0.1378, -0.3889],
        [ 0.5133,  0.3319,  0.6300,  0.5815, -0.0282]])
tensor([[0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1.],
        [0., 1., 1., 0., 1.]])
tensor([[-0.0000, -0.0000, -0.0000, -1.8244, -2.5004],
        [ 1.6065, -0.4142,  0.0000,  0.2757, -0.7778],
        [ 0.0000,  0.6638,  1.2600,  0.0000, -0.0563]])


4. Residual Connection

![](https://i.imgur.com/7Gt22WB.png)


[nice blog](https://towardsdatascience.com/what-is-residual-connection-efb07cab0d55)

In [73]:
class ResidualConnection:
    def __init__(self, block):
        self.block = block

    def __call__(self, x):
        return x + self.block(x)


torch.manual_seed(1337)
x = torch.randn(3, 20)
block = nn.Linear(20, 20)  # note: output shape must be the same as an input
residual = ResidualConnection(block)
residual(x).shape

torch.Size([3, 20])

Well, basically that's it. We've done it!!! 🎉


For the final version check scripts (or visit [this repo](https://github.com/karpathy/ng-video-lecture)) 😍


Cheers 🥂